In [1]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

# Read csv

In [2]:
# Feb data
zip_dir  = "../data/new_dfs2"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfors.append(df)
    

In [5]:
# Mar data
zip_dir  = "../data/new_dfs"
csvfiles = glob(os.path.join(zip_dir, 'dfrt*'))

dfrts = []
for csvfile in csvfiles:
    df1 = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfrts.append(df1)
    

In [18]:
dfrts[0]

,user_id,date,tweet,RT_flag,year,month,day,hour,time
0,4218274875,2020-02-29,RT @tama_punpa: ドラッグストア店員の思いです。私たちも同じ人間です。ありがと...,True,2020,2,29,14,14:59:58
1,907574528850616322,2020-02-29,RT @vote0611: SixTONESのマネさん大変だろうな歌が上手いから音楽方面も極...,True,2020,2,29,14,14:59:58
2,158072422,2020-02-29,RT @amneris84: 頻繁な会食を「いけないことなのか」と安倍首相。平時ならいいでし...,True,2020,2,29,14,14:59:58
3,861437073504452608,2020-02-29,RT @tama_punpa: ドラッグストア店員の思いです。私たちも同じ人間です。ありがと...,True,2020,2,29,14,14:59:58
4,1212404675275714562,2020-02-29,RT @uni_mafumafu: 【MV】人生逆転の神業／After the Rain【そ...,True,2020,2,29,14,14:59:58
...,...,...,...,...,...,...,...,...,...
1086400,1088041295124123650,2020-02-26,RT @YUElovekjj: キム・ジェジュンのやり取りの中で、他の芸能人を言い出すのは失...,True,2020,2,26,17,17:24:08
1086401,1581933457,2020-02-26,RT @nyato1005: ハピバ♡へなちょこしかかけんかった、ソリノエちゃん https...,True,2020,2,26,17,17:24:08
1086402,122111121,2020-02-26,RT @mas__yamazaki: 小川彩佳さん、もう番組内ではっきり言いましょうよ。「東...,True,2020,2,26,17,17:24:04
1086403,989763338417520641,2020-02-26,RT @pasubare0112: 眠剤効いてないのかな？眠れない。うーん……困った。眠いよ...,True,2020,2,26,17,17:24:01


# Filter V4 by 'エロい' - Day

In [3]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def filterV4(dfors, filetype, savedir, key_lang, filt_flag):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
            
    # Find tweets
    for i, dfor in tqdm(enumerate(dfors)):
         # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        # Add columns of keywords whose cell have 1 if this tweet includes a keyword 
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        d4 = dfor_c[dfor_c.V4 == 1]
        
        # Finally Connect dfs
        if i == 0:
            dfnew = d4
        else:
            dfnew = pd.concat([dfnew, d4], axis=0)
    
    
    # DF Only V4
    d4 = dfnew[dfnew.V4 == 1].drop(['V1', 'V2', 'V3', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20'], axis = 1)
    if filt_flag:
        # Remove 'エロい'
        d4_without_eroi = d4[~d4['tweet'].str.contains('エロい', na = False)]
        outname = 'V4b.csv'
    else:
        d4_without_eroi = d4
        outname = 'V4a.csv'

     # Make rows
    uni_dates = d4_without_eroi['date'].tolist()
    uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
    rows = []
    for date in uni_dates:
        r = d4_without_eroi.groupby('date').get_group(date).V4.sum()
        rows.append(r)
    data = np.array([uni_dates, rows]).T
    # columns
    cols = ['date', 'V4']
    # Create DataFrame
    dft = pd.DataFrame(data, columns=cols).groupby('date').sum()
    
    # Save to csv
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    savename = os.path.join(outdir, outname)
    dft.to_csv(savename)
            
    return dft

In [4]:
# OR
filetypes = ['V']
for filetype in filetypes:
    d4_without_eroi = filterV4(dfors, filetype=filetype, savedir='../new_results2/orjp/day', key_lang='jp', filt_flag=False)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10it [01:02,  6.20s/it]


In [7]:
# RT
filetypes = ['V']
for filetype in filetypes:
    d4_without_eroi = filterV4(dfrts, filetype=filetype, savedir='../new_results2/rtjp/day', key_lang='jp', filt_flag=True)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10it [00:31,  3.20s/it]


In [15]:
d4_without_eroi

,V4
date,
2020-01-31,568
2020-02-01,2336
2020-02-02,2898
2020-02-03,2461
2020-02-04,2492
2020-02-05,2403
2020-02-06,2268
2020-02-07,2005
2020-02-08,2171


# Filter V4 by 'エロい' - Day

In [35]:
path = '../new_results2/orjp/day/'
a = pd.read_csv(path+'V4a.csv').rename(columns={'V4': 'V4b'})
b = pd.read_csv(path+'V4b.csv').rename(columns={'V4': 'V4b'})

d = pd.concat([a, b.V4b], axis =1, join='inner')
d.diff(axis=1)

,date,V4b,V4b
0,NaN,NaN,0.0
1,NaN,NaN,-2.0
2,NaN,NaN,0.0
3,NaN,NaN,-2.0
4,NaN,NaN,-1.0
5,NaN,NaN,0.0
6,NaN,NaN,-1.0
7,NaN,NaN,-1.0
8,NaN,NaN,-2.0
9,NaN,NaN,-4.0


In [29]:
d = pd.concat([a, b.V4b], axis =1, join='inner')
d.diff(axis=1)

In [28]:
d.diff(axis=1)

,date,V4b,V4b
0,NaN,NaN,0.0
1,NaN,NaN,0.0
2,NaN,NaN,-2.0
3,NaN,NaN,0.0
4,NaN,NaN,0.0
5,NaN,NaN,0.0
6,NaN,NaN,0.0
7,NaN,NaN,0.0
8,NaN,NaN,0.0
9,NaN,NaN,0.0
